Incorporar modelos  más  avanzados  para  clasificar  texto,  e.g.  redes  neuronales recurrentes  con 
atención, BERT o GPT-3. 

Modelo BERT
El nombre completo de BERT es Representación de codificador bidireccional de transformadores, es decir, codificador de transformador bidireccional. La principal innovación del modelo es el método de pre-entrenamiento, que se utilizaMasked LMconNext Sentence PredictionLos dos métodos capturan la representación a nivel de palabra y oración, respectivamente. Como sustituto de Word2Vec, sus 11 direcciones en el campo de la PNL han mejorado enormemente su precisión. Se puede decir que es una tecnología innovadora óptima para redes auto residuales en los últimos años.

Las principales características del modelo Bert:

1. La arquitectura de red de BERT utiliza las múltiples capas propuestas en "Atención es todo lo que necesita"TransformerEstructura, su principal característica es abandonar los RNN y CNN tradicionales, a través del mecanismo de Atención para convertir la distancia de dos palabras en cualquier posición a 1, resolver de manera efectiva el difícil problema de dependencia a largo plazo en PNL, puede capturar más a fondo la oración Relación bidireccional.


2. Objetivos de entrenamiento de tareas múltiples usando el Modelo de lenguaje de máscara (MLM) [Wilson L Taylor. 1953. procedimiento de cloze: una nueva herramienta para medir la legibilidad.] Y Predicción de la siguiente oración (NSP);

3. Al usar máquinas más potentes para entrenar datos a mayor escala, los resultados de BERT alcanzaron un nivel completamente nuevo, y Google Open obtuvo el BERT multilingüe capacitadoCódigo modelo, Los usuarios pueden usar directamente BERT como la matriz de conversión de Word2Vec y aplicarlo eficientemente a sus propias tareas.

In [1]:
import pandas as pd

import tarfile
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

train_df = pd.read_csv('data/yelp_review_polarity_csv/train.csv', header=None)
print(train_df.head())

test_df = pd.read_csv('data/yelp_review_polarity_csv/test.csv', header=None)
print(test_df.head())

   0                                                  1
0  1  Unfortunately, the frustration of being Dr. Go...
1  2  Been going to Dr. Goldberg for over 10 years. ...
2  1  I don't know what Dr. Goldberg was like before...
3  1  I'm writing this review to give you a heads up...
4  2  All the food is great here. But the best thing...
   0                                                  1
0  2  Contrary to other reviews, I have zero complai...
1  1  Last summer I had an appointment to get new ti...
2  2  Friendly staff, same starbucks fair you get an...
3  1  The food is good. Unfortunately the service is...
4  2  Even when we didn't have a car Filene's Baseme...


In [2]:
train_df[0] = (train_df[0] == 2).astype(int)
test_df[0] = (test_df[0] == 2).astype(int)

In [3]:
print(train_df.head())
print(test_df.head())

   0                                                  1
0  0  Unfortunately, the frustration of being Dr. Go...
1  1  Been going to Dr. Goldberg for over 10 years. ...
2  0  I don't know what Dr. Goldberg was like before...
3  0  I'm writing this review to give you a heads up...
4  1  All the food is great here. But the best thing...
   0                                                  1
0  1  Contrary to other reviews, I have zero complai...
1  0  Last summer I had an appointment to get new ti...
2  1  Friendly staff, same starbucks fair you get an...
3  0  The food is good. Unfortunately the service is...
4  1  Even when we didn't have a car Filene's Baseme...


ID de fila, etiqueta de fila, letra única, texto que queremos clasificar

In [4]:
bert_df = pd.DataFrame({
    'id': range(len(train_df)),
    'label': train_df[0],
    'alpha': ['q']*train_df.shape[0],
    'text': train_df[1].replace(r'\n', ' ', regex=True)
})

train_bert_df, dev_bert_df = train_test_split(bert_df, test_size=0.01)

In [5]:
print(train_bert_df.head())

            id  label alpha                                               text
222231  222231      1     q  I love Egg Works, after a night of partying, i...
12025    12025      1     q  Fast friendly service, good food, no complaint...
493005  493005      1     q  I love this location.  I have had many issues ...
324158  324158      0     q  I've never been a thrift shopper but since ren...
194500  194500      0     q  Just saw Jack Johnson.  Great performance!   \...


BERT solo espera dos columnas: ID de fila, texto

In [6]:
test_bert_df = pd.DataFrame({
    'id': range(len(test_df)),
    'text': test_df[1].replace(r'\n', ' ', regex=True)
})

In [9]:
print(test_bert_df.head())

   id                                               text
0   0  Contrary to other reviews, I have zero complai...
1   1  Last summer I had an appointment to get new ti...
2   2  Friendly staff, same starbucks fair you get an...
3   3  The food is good. Unfortunately the service is...
4   4  Even when we didn't have a car Filene's Baseme...


Guardar los archivos 

In [8]:
train_bert_df.to_csv('data/train.tsv', sep='\t', index=False, header=False)
dev_bert_df.to_csv('data/dev.tsv', sep='\t', index=False, header=False)
test_bert_df.to_csv('data/test.tsv', sep='\t', index=False, header=False)

# EN TERMINAL  PARA EL MODELO 

python run_classifier.py --task_name=cola --do_train=true --do_eval=true --data_dir=./data/ --vocab_file=./uncased_L-12_H-768_A-12/vocab.txt --bert_config_file=./uncased_L-12_H-768_A-12/bert_config.json --init_checkpoint=./uncased_L-12_H768_A-12/bert_model.ckpt.index --max_seq_length=128 --train_batch_size=32 --learning_rate=2e-5 --num_train_epochs=3.0 --output_dir=./model_output --do_lower_case=False

# EN TERMINAL PARA PREDICCIÓN 
python run_classifier.py --task_name=cola --do_predict=true --data_dir=./data --vocab_file=./uncased_L-12_H-768-A-12/bert_config.json --init_checkpoint=./model_output/model.ckpt-<highest checkpoint number> --max_seq_length=128 --output_dir=./model_output